In [1]:
import sympy as sym
from os import path
import sympy.physics.mechanics as me
from sympy.abc import t
import moyra as ma
me.mechanics_printing()

In [2]:
DoFs = 1
# heave and fold angle as degrres of freedom
p = ma.DynamicModelParameters(DoFs)
p.DoFs  = ma.ModelSymbol(value = DoFs,string = 'DoFs')
#gravity vector
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.g_v = ma.ModelMatrix(value =[0,0,-1],length=3,string='g_v') # The stationary point
p.alpha = ma.ModelSymbol(value = 0,string = 'alpha')                     # AoA
p.beta = ma.ModelSymbol(value = 0,string = 'beta')                     # SideSlip
p.v_c = ma.ModelMatrix(value =[0,0,0],length=3,string='v_c') # The stationary point
p.v_hc = ma.ModelMatrix(value =[0,0,0],length=3,string='v_hc') # The stationary point
p.v_h = ma.ModelMatrix(value =[0,0,0],length=3,string='v_h') # The stationary point
p.m = ma.ModelSymbol(value = 0,string = 'm') # The stationary point
p.m_h = ma.ModelSymbol(value = 0,string = 'm_h') # The stationary point
p.I_xxh = ma.ModelSymbol(value = 0,string = 'I_xxh') # The stationary point
p.I_xx = ma.ModelSymbol(value = 0,string = 'I_xx') # The stationary point
p.I_yy = ma.ModelSymbol(value = 0,string = 'I_yy') # The stationary point
p.I_zz = ma.ModelSymbol(value = 0,string = 'I_zz') # The stationary point
p.Lambda = ma.ModelSymbol(value = 0,string = 'Lambda') # The stationary point
p.Wrench = ma.ModelMatrix(value =[0]*6,length=6,string='W') # The stationary point

y_0,x_0,z_0 = sym.symbols('y_0,x_0,z_0')
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')

In [3]:
hinge_frame = ma.frames.HomogenousFrame().R_z(p.beta).R_y(p.alpha).Translate(*p.v_h).R_z(p.Lambda).R_x(-p.q[0]).simplify()
fwt_frame = hinge_frame.R_z(-p.Lambda).simplify()
fwt_force_frame = fwt_frame.Translate(x_0,y_0,z_0)

body_frame = ma.frames.HomogenousFrame().R_z(p.beta).R_y(p.alpha).simplify()
gravity_frame =  ma.frames.HomogenousFrame().R_z(p.beta).R_y(p.alpha)

In [4]:
#Create Elemnts
M = ma.elements.MassMatrix(p.m,p.I_xx,p.I_yy,0,0,0,0)
M_h = ma.elements.MassMatrix(p.m_h,p.I_xxh,0,0,0,0,0)
## FWT mass element
fwt_ele = ma.elements.RigidElement(p.q, fwt_frame, M, grav_vec=p.g_v*p.g, com_pos=p.v_c, name='fwt',simplify = False)
#Hinge Mass element
hinge_ele = ma.elements.RigidElement(p.q, hinge_frame, M_h, grav_vec=p.g_v*p.g, com_pos=p.v_hc, name='hinge_mass',simplify = False)

In [5]:
fwt_force = ma.forces.ExternalForce.body_force(p.q,fwt_force_frame,F_x,F_y,F_z,M_x,M_y,M_z,simplify = False)

In [6]:
sm = ma.SymbolicModel.FromElementsAndForces(p.q,[fwt_ele,hinge_ele])

Generating EoM for Element 1 out of 2 - fwt:RigidElement
Generating EoM for Element 2 out of 2 - hinge_mass:RigidElement


In [7]:
funcs = []
fwt_pos = [x_0,y_0,z_0]
funcs.append(('get_fwt_V_b',fwt_ele.frame.A.T*fwt_ele.frame.transform_point([x_0,y_0,z_0]).diff(t)))
funcs.append(('get_pos_fwt',fwt_ele.frame.transform_global_point([x_0,y_0,z_0])))
funcs.append(('get_fwt_V_b_body',body_frame.A.T*fwt_ele.frame.transform_point([x_0,y_0,z_0]).diff(t)))
funcs.append(('get_fwt_A',fwt_ele.frame.A))
funcs.append(('get_body_A',body_frame.A))

In [8]:
#create function for an arbitary force of FWT
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')
W = sym.Matrix([F_x,F_y,F_z,M_x,M_y,M_z])
fwt_Q_tmp = fwt_ele.frame.Translate(*fwt_pos).BodyJacobian(p.q).T*W
idx = [i for i, e in enumerate(p.q) if e in fwt_ele.q]
fwt_Q = sym.zeros(len(p.q),1)
for i,i_fwt in zip(idx,fwt_ele._idx):
    fwt_Q[i] += fwt_Q_tmp[i_fwt]
funcs.append(('get_fwt_Q',fwt_Q))

In [9]:
#create function for a moment about the hinge
M_h = sym.symbols('M_h')
fwt_Q = sym.zeros(len(p.q),1)
fwt_Q[0] = M_h
funcs.append(('get_hinge_Q',fwt_Q))

In [10]:
funcs.append(('get_fold',p.q[0]))
funcs.append(('get_fold_dt',p.qd[0]))

In [11]:
base_dir = "D:\\AlphaBetaData\\EoM"
class_name = "OneDof_RC_fwt"
base_class = "mbd.BaseRC"
sm.to_matlab_class(p,base_dir,class_name,base_class,additional_funcs=funcs)

Unknown variable y_0 found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable z_0 found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable x_0 found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable y_0 found in function get_pos_fwt. It will be added to the function signature.
Unknown variable z_0 found in function get_pos_fwt. It will be added to the function signature.
Unknown variable x_0 found in function get_pos_fwt. It will be added to the function signature.
Unknown variable y_0 found in function get_fwt_V_b_body. It will be added to the function signature.
Unknown variable z_0 found in function get_fwt_V_b_body. It will be added to the function signature.
Unknown variable x_0 found in function get_fwt_V_b_body. It will be added to the function signature.
Unknown variable F_x found in function get_fwt_Q. It will be added to the function signature.
Unknown variable M_z found 